In [1]:
import numpy as np
import random

def generate_episode(P, R, policy, n_states, max_steps=100):
    episode = []
    state = random.choice(range(n_states))
    for _ in range(max_steps):
        action = policy[state]
        next_state = np.random.choice(range(n_states), p=P[action, state])
        reward = R[action, state]
        episode.append((state, action, reward))
        state = next_state
        if state == next_state:  # Assuming episode ends when reaching a terminal state
            break
    return episode

def monte_carlo_control(P, R, n_states, n_actions, gamma=0.9, epsilon=0.1, episodes=1000):
    Q = np.zeros((n_states, n_actions))
    returns = { (s, a): [] for s in range(n_states) for a in range(n_actions) }
    policy = np.zeros(n_states, dtype=int)

    for _ in range(episodes):
        episode = generate_episode(P, R, policy, n_states)
        G = 0
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = gamma * G + reward
            if not any((state == x[0] and action == x[1]) for x in episode[:t]):
                returns[(state, action)].append(G)
                Q[state, action] = np.mean(returns[(state, action)])
                policy[state] = np.argmax(Q[state])

    return policy, Q

### Utility Functions ###

def validate_transition_matrix(P):
    assert np.allclose(P.sum(axis=2), 1), "Transition probabilities must sum to 1."

def validate_reward_matrix(R, P):
    assert R.shape == P.shape[:2], "Reward matrix dimensions must match the transition matrix."

def generate_random_mdp(n_states, n_actions):
    P = np.zeros((n_actions, n_states, n_states))
    for a in range(n_actions):
        for s in range(n_states):
            P[a, s, :] = np.random.dirichlet(np.ones(n_states))
    R = np.random.rand(n_actions, n_states)
    return P, R

### Example Usage ###

# Generate a random MDP
n_states = 3
n_actions = 2
P, R = generate_random_mdp(n_states, n_actions)

# Validate the MDP
validate_transition_matrix(P)
validate_reward_matrix(R, P)

# Solve the MDP using Monte Carlo Control
policy_mc, Q_mc = monte_carlo_control(P, R, n_states, n_actions)
print("Optimal Policy (Monte Carlo Control):", policy_mc)
print("Q-Table (Monte Carlo Control):", Q_mc)

Optimal Policy (Monte Carlo Control): [0 0 0]
Q-Table (Monte Carlo Control): [[0.43151488 0.        ]
 [0.08937007 0.        ]
 [0.56597802 0.        ]]
